# 1. AWS Bedrock 기초 모델을 사용한 프로토타입
CML 핸즈온 랩의 첫 번째 연습에 오신 것을 환영합니다. 이 노트북에서는 외부에서 호스팅되는 기초 모델을 호출하는 방법에 대해 알아봅니다. 이 연습에서는 AWS Bedrock 서비스와 거기에 호스팅된 ~Anthropic Claude~ Mistral 모델을 사용합니다.

![image](../assets/jupypter-session-bedrock.png)

### 1.1 Cloudera CML 
여기서 주목할 점은 AWS 서비스와 상호 작용하는 `boto3` SDK입니다. `get_bedrock_client` 함수는 AWS의 [github 저장소](https://github.com/aws-samples/amazon-bedrock-workshop/blob/109ed616fd14c9eb26eda9bef96eb78c490d5ef6/utils/bedrock.py#L13)에서 가져온 것입니다. 자신의 환경에서 이 코드를 실행하는 경우 AWS 키를 환경 변수로 설정하는 것이 좋습니다.

In [5]:
import json
import os
from typing import Optional
import boto3
from botocore.config import Config

if os.environ.get("AWS_ACCESS_KEY_ID") == "":
    os.environ["AWS_ACCESS_KEY_ID"] = "<YOUR-ACCESS-KEY-ID>"   # Replace this if running in your own environment

if os.environ.get("AWS_SECRET_ACCESS_KEY") == "":
    os.environ["AWS_SECRET_ACCESS_KEY"] = "<YOUR-SECRET-ACCESS-KEY>"   # Replace this if running in your own environment

if os.environ.get("AWS_FOUNDATION_MODEL_ID") == "":
    os.environ["AWS_FOUNDATION_MODEL_ID"] = "<FOUNDATION-MODEL-ID>"   # Replace this if running in your own environment

# TODO: for a lab, can reduce some of the checks in the below function
def get_bedrock_client(
    assumed_role: Optional[str] = None,
    endpoint_url: Optional[str] = None,
    region: Optional[str] = None,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    endpoint_url :
        Optional override for the Bedrock service API Endpoint. If setting this, it should usually
        include the protocol i.e. "https://..."
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if endpoint_url:
        client_kwargs["endpoint_url"] = endpoint_url

    bedrock_client = session.client(
        service_name="bedrock-runtime",
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

그런 다음 클라이언트가 초기화되어 Bedrock 서비스를 사용할 수 있는 AWS 리전에 바인딩됩니다. [2023년 10월 현재](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-bedrock-asia-pacific-tokyo-aws-region/) 이러한 리전은 us-east-1, us-west-2 및 ap-northeast-1입니다. 기본값으로 `us-east-1`을 사용합니다. 이는 환경 변수로 덮어쓸 수 있습니다.

In [7]:
# Initializing the bedrock client using AWS credentials
# If you are using a special Assumed role or custom endpoint url, see get_bedrock_client
if os.environ.get("AWS_DEFAULT_REGION") == "":
    os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

boto3_bedrock = get_bedrock_client(region=os.environ.get("AWS_DEFAULT_REGION", None))

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


### 1.3 원하는 지침(Instruction) 설정: Text Summarization
이 노트북에 표시된 기반 모델(~Anthropic의 Claude~ Mistral)은 일반적인 instruction-following text 생성 모델입니다. 즉, 제공된 지침을 따르는 응답을 생성하기 위해 몇 가지 지침과 입력 텍스트를 제공할 수 있습니다. 예를 들어, 텍스트의 일부를 몇 개의 요점으로 요약하는 지침을 기초 모델에 제공합니다. 모델 지침은 일반적으로 규정된 패턴을 따르며 사용된 모델에 따라 달라집니다. 즉, 다른 모델에 지침을 제공하는 표준적인 방법은 없습니다. 

In [8]:
instruction_text = """Human: Please provide a summary of the text inside <text></text> XML tags. Do not add any information that is not mentioned in this text. 
                             Provide no more than 3 bullet points in the summary, each being a complete sentece. 
                             Start your summary with simply saying "Here's a brief summary of the provided text:". 
                    <text>{{USER_TEXT}}</text>
                    Assistant:"""

### 1.4 입력 텍스트 설정 및 완전한 프롬프트 생성 <a id='1.4'></a>
아래는 요약하고자 하는 입력 텍스트입니다. 이 텍스트의 길이와 포함된 지침(Instruction)은 선택한 모델의 context window size에 맞아야 합니다. 클로드의 경우 약 9,000단어입니다.

In [9]:
input_text = '''Machine learning has become one of the most critical capabilities for modern businesses to grow and stay competitive today. From automating internal processes to optimizing the design, creation, and marketing processes behind virtually every product consumed, ML models have permeated almost every aspect of our work and personal lives.
ML development is iterative and complex, made even harder because most ML tools aren’t built for the entire machine learning lifecycle. Cloudera Machine Learning on Cloudera Data Platform accelerates time-to-value by enabling data scientists to collaborate in a single unified platform that is all inclusive for powering any AI use case. Purpose-built for agile experimentation and production ML workflows, Cloudera Machine Learning manages everything from data preparation to MLOps, to predictive reporting. Solve mission critical ML challenges along the entire lifecycle with greater speed and agility to discover opportunities which can mean the difference for your business.
Each ML workspace enables teams of data scientists to develop, test, train, and ultimately deploy machine learning models for building predictive applications all on the data under management within the enterprise data cloud. ML workspaces support fully-containerized execution of Python, R, Scala, and Spark workloads through flexible and extensible engines.'''

# Replace instruction placeholder to build a complete prompt
full_prompt = instruction_text.replace("{{USER_TEXT}}", input_text)

### 1.5 LLM 모델에 대한 API 요청 생성
Bedrock으로 보내 처리할 JSON 페이로드를 생성합니다. 이 API 요청에 필요한 매개변수와 형식은 모델에 따라 다릅니다. AWS Bedrock 설명서를 참조하세요.

In [10]:
os.environ["AWS_FOUNDATION_MODEL_ID"]

'mistral.mistral-7b-instruct-v0:2'

In [11]:
# Model expects a JSON object with a defined schema
if 'claude' in os.environ["AWS_FOUNDATION_MODEL_ID"].lower():
    body = json.dumps({"prompt": full_prompt,
             "max_tokens_to_sample":4096,
             "temperature":0.6,
             "top_k":250,
             "top_p":1.0,
             "stop_sequences":[]
              })
elif 'mistral' in os.environ["AWS_FOUNDATION_MODEL_ID"].lower():
    body = json.dumps({"prompt": full_prompt,
             "max_tokens":200,
             "temperature":0.5,
             "top_p":0.9,
             "top_k":50,
              })



# Provide a model ID and call the model with the JSON payload
modelId = os.environ["AWS_FOUNDATION_MODEL_ID"]
response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept='application/json', contentType='application/json')
response_body = json.loads(response.get('body').read())
print("Model results successfully retreived")

Model results successfully retreived


### 1.6 결과 검토

In [12]:
result = response_body.get('outputs')

In [13]:
if 'claude' in os.environ["AWS_FOUNDATION_MODEL_ID"].lower():
    result = response_body.get('completion')
elif 'mistral' in os.environ["AWS_FOUNDATION_MODEL_ID"].lower():
    result = response_body.get('outputs')
print(result)

[{'text': " Here's a brief summary of the provided text:\n1. Machine learning is a vital capability for businesses to grow and compete, influencing various aspects of work and personal life.\n2. Cloudera Machine Learning on Cloudera Data Platform expedites the machine learning lifecycle, offering a unified platform for data scientists to collaborate.\n3. This platform, purpose-built for ML workflows, manages data preparation to MLOps and predictive reporting, enabling faster solution of ML challenges.", 'stop_reason': 'stop'}]


**(보너스)** [1.4단계](#1.4)로 돌아가서 모델이 요약할 다른 텍스트를 붙여넣습니다. 작업에서 어떻게 되는지 확인하세요.

####  주요 사항

* [Cloudera Machine Learning](https://docs.cloudera.com/machine-learning/cloud/product/topics/ml-product-overview.html#cdsw_overview)은 타사 기반 모델과 통합할 수 있는 유연한 환경을 제공합니다.
* JupyterLabs는 지원되는 편집기이며, [사용자 정의 런타임](https://docs.cloudera.com/machine-learning/cloud/runtimes/topics/ml-creating-a-customized-runtimes-image.html)에 선택적으로 추가할 수 있는 다른 편집기도 있습니다(예: RStudio, VSCode)
* 사용자는 가장 효율적인 개발 도구를 사용하여 LLM 솔루션을 빠르게 프로토타입화할 수 있습니다.

### 1.7 prototype_with_cml_model.ipynb 파일을 열어 CML 환경에서 배포된 LLM 모델을 호출해 보세요